In [107]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE


url = 'https://drive.google.com/file/d/15X00ZWBjla7qGOIW33j8865QdF89IyAk/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
df = pd.read_csv(path)
titles = df['title'][0:10000] # Taken only 10000 samples as Colab was Crashing a lot


tokenizer = Tokenizer()



In [82]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import heapq
tokens = [word_tokenize(t) for t in titles]
stop_words = set(stopwords.words('english')) # Get rid of Stopwords



In [84]:
word2count = {}      # Create Bag of words model for the text data
for data in titles:
    words = nltk.word_tokenize(data)
    for t in words:
      if t.lower() in stop_words:
       words.remove(t)
    for word in words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1
freq_words = heapq.nlargest(200, word2count, key=word2count.get)

X = []
for data in titles:
    vector = []
    for word in freq_words:
        if word in nltk.word_tokenize(data):
            vector.append(1)
        else:
            vector.append(0)
    X.append(vector)
X = np.asarray(X)

In [111]:
Y_ = df.up_votes[0:10000] 
#Train a Xgboost model to predict upvotes 
train_X, test_X, train_y, test_y = train_test_split(X, Y_, test_size = 0.15)
  
xgb_r = xg.XGBRegressor(n_estimators = 60 , max_depth =15)
  
xgb_r.fit(train_X, train_y)


[11:26:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=15, min_child_weight=1, missing=None, n_estimators=60,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [112]:
pred = xgb_r.predict(test_X)
  
# RMSE Computation
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  37.838332


So this is not a great model , probably textual information cannot be a great predictor for the number of Upvotes
